## Parte 1 :   Importando Bibliotecas que serão usadas

In [94]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv

import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize
from nltk import tokenize 
from nltk.stem import RSLPStemmer
from nltk.stem.porter import *
import heapq
import time
%matplotlib inline


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!



## Lendo arquivo csv
## Eliminando caracteres com tamanho 2 ou menor e deixando todos em caixa baixa




In [95]:
arquivo = pd.read_csv('results.csv')
texto = arquivo['text']

def eliminar_carac(text):
  return re.sub(r'(\W+|\d+|\b\w{0,2}\b)', ' ', text.lower())
texto = texto.apply(eliminar_carac)

texto = texto.apply(tokenize.word_tokenize, language='portuguese')
palavras = []


texto.head()

0    [documento, páginas, vazado, onu, calcula, que...
1    [volta, israel, onde, fez, parte, comitiva, pa...
2    [ditador, nicolás, maduro, não, despacha, palá...
3    [steve, bannon, estrategista, donald, trump, d...
4    [entre, março, º, abril, pessoas, foram, presa...
Name: text, dtype: object

## Eliminando as stopwords : Usando a biblioteca nltk
## Algoritmo Build Index : Usada para uma indexação simples


In [96]:
stopwords = nltk.corpus.stopwords.words('portuguese')

def eliminar_stopwords(text):
  palavras = []
  for palavra in text:
    if(palavra not in stopwords):
      palavras.append(palavra)
  return palavras 

def build_index(arquivo):
  numero_da_lista = 0
  dic_index = {}
  
  for documento in arquivo:
    
    numero_da_lista += 1
    doc = eliminar_stopwords(documento)
    
    for palavra in doc:
      ocorrencia = doc.count(palavra)
      if palavra not in dic_index:
        dic_index[palavra] = []
      par = (numero_da_lista, ocorrencia)
      if par not in dic_index[palavra]:
         dic_index[palavra].append(par)
  return dic_index

index = build_index(texto)
tabela = {'palavra' : list(index.keys()),
       'doc|ocorrencia' : list(index.values()) }
df = pd.DataFrame(tabela)
df.head()
# df.to_csv('index.csv') salvar o arquivo em csv



,palavra,doc|ocorrencia
0,documento,"[(1, 1), (26, 1), (58, 1), (80, 1)]"
1,páginas,"[(1, 1), (26, 1), (58, 1), (80, 1)]"
2,vazado,"[(1, 1), (26, 1), (58, 1), (80, 1)]"
3,onu,"[(1, 1), (6, 1), (26, 1), (34, 1), (58, 1), (6..."
4,calcula,"[(1, 1), (26, 1), (58, 1), (80, 1)]"


# Parte 2 : Implementação do processamento de consulta documento-por-vez e termo-por-vez



## Documento Por Vez : Calcula escores completos para documentos processando todas as listas de termos, um documento por vez

In [0]:
def document_por_vez(Q, I, k):
    L = []
    R = []
    for word in Q.split(" "):
        if word in I.keys():
            L.append(I[word])
    for document in range(1, len(texto)+1):
        sd = 0
        for l in L:
            for i in l:
                if (i[0] == document):
                    sd += i[1]
                    break
        if (sd != 0):
          heapq.heappush(R, (sd, document))
    return heapq.nlargest(k, R)

## Termo Por Vez: Acumula escores para documentos processando listas de termos, um termo por vez

In [0]:
def termo_por_vez(Q, I, k):
    A = {}
    L = []
    R = []
    for word in Q.split(" "):
        if word in I.keys():
            L.append(I[word])
    for l in L:
        for post in l:
            d = post[0]
            freq = post[1]
            if (d in A.keys()):
                A[d] += freq
            else:
                A[d] = freq
    for (d, ad) in A.items():
        sd = ad
        heapq.heappush(R, (sd, d))
    return heapq.nlargest(k, R)

##Definindo consultas de um termo 

In [0]:
queries = ["política", "presidente", "bolsonaro", "maduro", "onu"]

## Executa 5 consultas em cada algoritmo retornando os top-10 

In [0]:
def compare_doc_term(queries):  
  # Documento
  result_doc = []
  time_doc = []
  # Termo
  result_term = []
  time_term = []
  k = 10
  for query in queries:
    # Documento por vez
    inicio_doc = time.time()
    result_doc.append(document_por_vez(query, index, k))
    fim_doc = time.time()
    time_doc.append(fim_doc - inicio_doc)
    # Termo Por Vez
    inicio_term = time.time()
    result_term.append(termo_por_vez(query, index, k))
    fim_term = time.time()
    time_term.append(fim_term - inicio_term)


  tabela = {
      'query': queries,
      'documento_por_vez' : results_document,
      'termo_por_vez' : results_term,
      'comparando' : queries_df.documento_por_vez == queries_df.termo_por_vez
    
  }
  return tabela

## Resultado Top-10 documentos

In [147]:
queries_df = pd.DataFrame(compare_doc_term(queries))
queries_df
# queries_df.to_csv('queries.csv') salvar o arquivo em csv

,query,documento_por_vez,termo_por_vez,comparando
0,política,"[(1, 105), (1, 82), (1, 54)]","[(1, 105), (1, 82), (1, 54)]",True
1,presidente,"[(1, 113), (1, 107), (1, 77), (1, 55), (1, 24)...","[(1, 113), (1, 107), (1, 77), (1, 55), (1, 24)...",True
2,bolsonaro,"[(1, 121), (1, 119), (1, 117), (1, 115), (1, 1...","[(1, 121), (1, 119), (1, 117), (1, 115), (1, 1...",True
3,maduro,"[(1, 111), (1, 98), (1, 78), (1, 76), (1, 66),...","[(1, 111), (1, 98), (1, 78), (1, 76), (1, 66),...",True
4,onu,"[(1, 80), (1, 79), (1, 63), (1, 58), (1, 34), ...","[(1, 80), (1, 79), (1, 63), (1, 58), (1, 34), ...",True


## Comparando os tempos médios de execução

In [148]:
tabela_comparar = {
    'tempo médio documento': time_results_document,
    'tempo médio termo' : time_results_term
}

tabela_df = pd.DataFrame(tabela_comparar)
tabela_df

,tempo médio documento,tempo médio termo
0,0.000099,0.000010
1,0.000077,0.000007
2,0.000283,0.000040
3,0.000098,0.000009
4,0.000098,0.000009


##Chegando a conclusão que o tempo medio do algoritmo termo-por-vez  é menor que documento-por-vez 

#Parte 3 :  Consulta conjuntiva